In [2]:
from bs4 import BeautifulSoup as bs
import os
import pandas as pd
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import glob
import seaborn as sns
import plotly.express as px
import descartes
import re

In [3]:
all_files = glob.glob("temp/*.csv")

ldf = []

for file in all_files:
    ldf.append(pd.read_csv(file))
    
forumDF = pd.concat(ldf, axis=0, ignore_index=True)
forumDF.to_csv('../saved_gen_df/forumDF_full_df.csv', index=None)

In [4]:
check_df = forumDF[['date', 'board', 'nReplies', 'nViews']].groupby(by='date').describe().reset_index().set_index('date')

In [5]:
check_df = check_df.set_axis([f"{x}{y}" for x, y in test.columns], axis=1, inplace=False)

NameError: name 'test' is not defined

In [6]:
check_df['s'] = forumDF[['date','nReplies']].groupby(by='date').sum()

In [7]:
def rmSmallWords(s):
    return re.sub(r'\b\w{1,3}\b', '', str(s))

def rmStopWords(xs):
    return [word for word in xs if word not in stopwords.words('english')]

In [8]:
forumDF['title'] = forumDF['title'].astype(str)
forumDF['clean_title'] = forumDF['title'].str.replace("[^a-zA-Z#]", " ")
forumDF['clean_title'] = forumDF['clean_title'].apply(rmSmallWords)
forumDF['clean_title'] = forumDF['clean_title'].str.lower()

In [9]:
import nltk
from nltk.corpus import stopwords

forumDF['tokens'] = forumDF['clean_title'].apply(lambda x: x.split()) # tokenizing
from nltk.stem.porter import *
stemmer = PorterStemmer()

forumDF['tokens']  = forumDF['tokens'].apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
forumDF['tokens_filtered'] = forumDF['tokens'].apply(rmStopWords)

In [10]:
forumDF['tokens_filtered'] = [word for word in forumDF['tokens'] if word not in stopwords.words('english')]

In [14]:
subdf_bef = forumDF.drop(columns=['Unnamed: 0', 'title'])[(forumDF['date'] >= '2014-10-05') & (forumDF['date'] <= '2014-11-06')]
subdf_af = forumDF.drop(columns=['Unnamed: 0', 'title'])[(forumDF['date'] >= '2014-11-06') & (forumDF['date'] <= '2014-12-05')]

In [12]:
subdf_bef.groupby(by=['date','board']).describe()

nReplies  \
                                                                 count   
date       board                                                         
2014-10-17 ('Community', 'Drug Safety')                           20.0   
           ('Community', 'Philosophy, Economics and Justice')     20.0   
           ('General Category', 'Access vendor-only forum')        1.0   
           ('General Category', 'Bugs')                          258.0   
           ('General Category', 'General Discussion')            432.0   
...                                                                ...   
2014-11-06 ('General Category', 'Vendor Discussion')             120.0   
           ('Local forums', 'German')                             20.0   
           ('Official section', 'Info')                            5.0   
           ('Official section', 'News')                            6.0   
           ('Offtopic', 'Generic randomness')                     20.0   

                                                                           \
                                                                     mean   
date       board                                                            
2014-10-17 ('Community', 'Drug Safety')                         13.600000   
           ('Community', 'Philosophy, Economics and Justice')   15.200000   
           ('General Category', 'Access vendor-only forum')      0.000000   
           ('General Category', 'Bugs')                          4.837209   
           ('General Category', 'General Discussion')           21.594907   
...                                                                   ...   
2014-11-06 ('General Category', 'Vendor Discussion')            56.591667   
           ('Local forums', 'German')                           31.800000   
           ('Official section', 'Info')                          5.000000   
           ('Official section', 'News')                         76.833333   
           ('Offtopic', 'Generic randomness')                  124.650000   

                                                                           \
                                                                      std   
date       board                                                            
2014-10-17 ('Community', 'Drug Safety')                         14.550818   
           ('Community', 'Philosophy, Economics and Justice')   24.771801   
           ('General Category', 'Access vendor-only forum')           NaN   
           ('General Category', 'Bugs')                          6.496082   
           ('General Category', 'General Discussion')          201.977533   
...                                                                   ...   
2014-11-06 ('General Category', 'Vendor Discussion')           185.054409   
           ('Local forums', 'German')                           47.228002   
           ('Official section', 'Info')                          8.660254   
           ('Official section', 'News')                         52.814455   
           ('Offtopic', 'Generic randomness')                  332.114006   

                                                                            \
                                                                min    25%   
date       board                                                             
2014-10-17 ('Community', 'Drug Safety')                         0.0   5.25   
           ('Community', 'Philosophy, Economics and Justice')   0.0   4.75   
           ('General Category', 'Access vendor-only forum')     0.0   0.00   
           ('General Category', 'Bugs')                         0.0   1.00   
           ('General Category', 'General Discussion')           0.0   1.00   
...                                                             ...    ...   
2014-11-06 ('General Category', 'Vendor Discussion')            0.0   0.00   
           ('Local forums', 'German')                           1.0   7.00   
         

In [ ]:
all_words_bfx = []
for board in subdf_bef['board'].unique():
    all_words_befx.append([board, sum(word_extract(subdf_bef['tokens_filtered']), [])])

In [ ]:
all_words_afx = []
for board in subdf_af['board'].unique():
    all_words_afx.append([board, sum(word_extract(subdf_af['tokens_filtered']), [])])

In [ ]:
subdf_af['board'].unique()

In [ ]:
np.array(all_words_afx).tofile('../saved_gen_df/all_words_1moaf', sep=",")
np.array(all_words_befx).tofile('../saved_gen_df/all_words_1mobf', sep=",")

In [ ]:
all_words = ' '.join([text for text in forumDF['clean_title'][(forumDF['date'] >= '2014-10-05') 
                                                        & (forumDF['date'] <= '2014-11-06') 
                                                   #    & (forumDF['board'] == "('General Category', 'General Discussion')" )
                                                             ]])

from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
def word_extract(x):
    hashtags = []
    # Loop over the words in the tweet
    for i in x:
        hashtags.append(i)

    return hashtags

HT_regular = word_extract(forumDF['tokens_filtered'][(forumDF['date'] >= '2014-11-06') 
                                                        & (forumDF['date'] <= '2014-12-05')])
HT_regular = sum(HT_regular,[])

In [ ]:
import nltk # for text manipulation

a = nltk.FreqDist(HT_regular)
d = pd.DataFrame({'Hashtag': list(a.keys()),
                  'Count': list(a.values())})

# selecting top 20 most frequent hashtags     
d = d.nlargest(columns="Count", n = 20) 
print(d)
plt.figure(figsize=(16,5))
ax = sns.barplot(data=d, x= "Hashtag", y = "Count")
ax.set(ylabel = 'Count')
plt.show()

In [ ]:
HT_bf = word_extract(forumDF['tokens_filtered'][(forumDF['date'] <= '2014-11-06') 
                                                        & (forumDF['date'] >= '2014-10-05')])
HT_bf = sum(HT_bf,[])

c = nltk.FreqDist(HT_bf)
e = pd.DataFrame({'Hashtag': list(c.keys()),
                  'Count': list(c.values())})

# selecting top 20 most frequent hashtags     
e = e.nlargest(columns="Count", n = 20) 
print(e)
plt.figure(figsize=(16,5))
ax = sns.barplot(data=e, x= "Hashtag", y = "Count")
ax.set(ylabel = 'Count')
plt.show()

In [ ]:

all_words = ' '.join([text for text in forumDF['title'][forumDF['board'] == "('General Category', 'General Discussion')"]])

from wordcloud import WordCloud
wordcloud = WordCloud(width=800, height=500, random_state=21, max_font_size=110).generate(all_words)

plt.figure(figsize=(10, 7))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.show()

In [ ]:
test = repliesAndViewsPerDateAndBoard = forumDF[['date', 'board', 'nReplies', 'nViews']].groupby(by=['date', 'board']).describe()

In [ ]:
test = test.set_axis([f"{x}{y}" for x, y in test.columns], axis=1, inplace=False)

In [ ]:
forumDF.board.unique()

In [ ]:
test

In [ ]:
fig, ax1 = plt.subplots( figsize=(20,10))

ax1.set_xlabel('date (d)')
ax1.set_ylabel('ptime Average per day for Agora Market')
ax1.plot(repliesAndViewsPerDateAndBoard.index, repliesAndViewsPerDateAndBoard[('nViews', 'count')])
ax1.tick_params(axis='y')
plt.show()

In [ ]:
gapminder = px.data.gapminder()
px.scatter(test.head(1000), x='nRepliesmax',
           y='nViewsmax', animation_frame='date', animation_group="board",
            
           size='nViewscount', color="board",range_x=[1,500000], range_y=[1, 500000], log_y=True, log_x=True)

In [ ]:
forumDF.groupby(by=['date']).describe()

In [ ]:
forum_stat = pd.read_csv('../saved_gen_df/forum_stats_df.csv')

In [ ]:
forum_stat= forum_stat.set_index('date')

In [ ]:
forum_stat

In [ ]:
test.tail(1000).max()

In [ ]:
print(test['nViewsmax'].max())
print(test['nRepliesmax'].max())
print(test['nViewsmax'].min())
print(test['nRepliesmax'].min())

In [ ]:
forumDF = pd.DataFrame(columns=['title', 'author','nReplies', 'nViews','lastPost', 'date', 'board'])
count_e = os.listdir('../data/agora-forums/2014-01-09/index.php')

for entry in os.scandir('../data/agora-forums/2014-01-09/index.php'):
    count = 0
    with open(entry.path) as fp:
        print(entry.path)
        soup = bs(fp, features="html.parser")
        if(len(soup.find_all('body'))>0):
            board = soup.find_all('body')[0].find_all('div', class_="navigate_section")
        else:
            print(entry.path)
        if(len(board[0].find_all('a'))>=3):
            (board0, board1) = (board[0].find_all('a')[1].text, board[0].find_all('a')[2].text)
        else:
            (board0, board1) = (None, None)
            print(len(board[0].find_all('a')))
        if(len(soup.find_all('tbody'))>0):
            topicList = soup.find_all('tbody')[0].find_all('tr')
            
    count += 1
    
print(count)